#### TP1 -- #Grupo 17

# Experimentación
En el presente cuaderno de Jupiter se corren los experimentos planteados y se escriben los resultados en archivos .CSV en la carpeta de resultados para luego ser analizados en el notebook de Analisis de Datos.

In [2]:
import math, subprocess
import pandas as pd
import numpy as np
import random
import filecmp
from IPython.display import display, clear_output

In [3]:
#Hacemos make para compilar
%%capture
subprocess.Popen(["make"], stdout=subprocess.PIPE, cwd="../src")

UsageError: Line magic function `%%capture` not found.


In [5]:
# funcion para correr experimento en C++
def correr_experimento(metodo, input, output):
     process = subprocess.Popen(["../src/tp1", input, output, metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)


In [6]:
# Corremos los tests de la catedra para hacer el analisis de dif absoluta
correr_experimento("0", "test_completos/test_completo_10_1.in", "test_completos/test_completo_10_11.out")
correr_experimento("0", "test_completos/test_completo_100_4.in", "test_completos/test_completo_10_11.out")
correr_experimento("0", "test_completos/test_completo_1000_8.in", "test_completos/test_completo_10_11.out")

In [7]:
# funcion para correr experimento con los 3 metodos 
def experimentar(input, output):
    for i in range(3):
        correr_experimento(str(i), f"{input}.dat", f"{output}_{i}.out")
    

In [10]:
# corremos los experimentos con los 3 metodos estudiados
experimentar("nba/nba_2016_scores","nba/nba_2016_scores")

## Experimento 1 -  Fuerza de los Oponentes:

Tenemos dos datasets adhoc, uno con una temporada perfecta para el primer equipo y otro con un upset . LUego estos seran analizados en detalles en el notebook Analisis de Experimentos


In [9]:
#Caso chico
experimentar("dataset_adhoc/perfect_season","dataset_adhoc/perfect_season")
experimentar("dataset_adhoc/upset_in_perfect_season","dataset_adhoc/upset_in_perfect_season")
#Caso grande NBA
experimentar("dataset_adhoc/nba_2016_scores","dataset_adhoc/nba_2016_scores")
#Se modifico esta linea 20151102 6 103 23 117 para que Philadelphia, el ultimo equipo le gane a uno de los primeros Cleveland
experimentar("dataset_adhoc/upset_nba_2016_scores","dataset_adhoc/upset_nba_2016_scores")


## Experimento 2 -  Relevancia del Orden de los Eventos:

Queremos ver si alterando la sequencia de partidos cambia el resultado

In [10]:
# Queremos crear un archivo con el orden invertido de los partidos.
def cambiar_orden(dir):
    file = open(f"{dir}.dat", "r")
    list = file.readlines()
    file.close()
    l1 = list[0]
    list = list[1:] 
    list.reverse()
    list = [l1] + list
    file = open(f"{dir}_reversed.dat", "w")
    for value in list:
        file.write(value)
    file.close()

    # y otro archivo con un orden aleatorio.
    file = open(f"{dir}.dat", "r")
    list = file.readlines()
    file.close()
    l1 = list[0]
    list= list[1:]
    random.shuffle(list)
    list = [l1] + list
    file = open(f"{dir}_random.dat", "w")
    for value in list:
        file.write(value)
    file.close() 

In [17]:
# funcion para comaparar si hubo cambios al modificar el orden
def comparar():
    flag_diferencias = False
    datasets = ["nba/nba_2016_scores",
                "nfl/nfl_2019","atp/atp_matches_2015","nhl/nhl_2019"]
    for d in datasets:
        cambiar_orden(d)
        #experimentamos con el dataset origininal
        experimentar(d,d)
        #luego con el invertido y el aleatorio
        experimentar(f'{d}_reversed',f'{d}_reversed')
        experimentar(f'{d}_random',f'{d}_random')
        for i in range(3):
            #comparamos si el resultado es el mismo
            dir1=f'{d}_{i}.out'
            dir2=f'{d}_reversed_{i}.out'
            if (not filecmp.cmp(dir1,dir2,shallow=False) ):
                flag_diferencias = True
                print("Diferencias encontradas en ", dir1)
    if(not flag_diferencias):
        print("No se encontraron diferencias")


In [12]:
comparar()

Diferencias encontradas en  nba/nba_2016_scores_2.out
Diferencias encontradas en  nfl/nfl_2019_2.out
Diferencias encontradas en  nhl/nhl_2019_2.out


## Experimento 3 -  Independencia de los partidos:

In [21]:
#Caso chico
experimentar("Exp3/test_1","Exp3/out/test_1.dat")
#Se modificaron valores arbitrarios, sin afectar a los equipos 1 y 2
experimentar("Exp3/test_1_mod1","Exp3/out/test_1_mod1")
experimentar("Exp3/test_1_mod2","Exp3/out/test_1_mod2")
experimentar("Exp3/test_1_mod3","Exp3/out/test_1_mod3")

#Caso grande NFL --------------------------------------
#MEJOR EQUIPO (NFC): San Francisco (id=28)
#1er PEOR EQUIPO (AFC): Cincinatti (id=7)
#2do PEOR EQUIPO (AFC): Miami (id=20)

#dataset original: nfl_2019.dat
experimentar("Exp3/nfl_2019","Exp3/out/nfl_2019")

#Se modifica linea 234: "20191222 20 38 7 35" por "20191222 20 35 7 38"
# a favor de Cincinatti

#Se espera que en Colley se sienta esa diferencia
experimentar("Exp3/nfl_2019_mod","Exp3/out/nfl_2019_mod")

## Experimento 4 -  Tiempo:

In [12]:
import datetime

columnas = ["dataset", "metodo","tiempo"];
filas = []
input = ["nba/nba_2016_scores.dat",
         "atp/atp_matches_2015.dat",
         "nfl/nfl_2019.dat",
         "nhl/nhl_2019.dat"]
metodos = ["colley","wp","sr"]
datasets = ["NBA 2016","ATP 2015","NFL 2019","NHL 2019"]

cant_exp = 10
for n_ds in range(len(input)):
    for n_met in range(len(metodos)):
        for i in range(cant_exp):
            fila_actual = []
            fila_actual.append(datasets[n_ds])
            fila_actual.append(metodos[n_met])

            init = datetime.datetime.now()
            correr_experimento(str(n_met), f"{input[n_ds]}", f"Exp4/{input[n_ds]}_{n_met}.out")
            end = datetime.datetime.now()
            time=end-init  
            fila_actual.append(str(time.microseconds))

            filas.append(fila_actual)
            #print(fila_actual)

resultado = pd.DataFrame(filas, columns=columnas);
resultado.to_csv(F"Exp4/resultado_exp4.csv", index=False, header=True);
display(resultado)

,dataset,metodo,tiempo
0,NBA 2016,colley,5780
1,NBA 2016,colley,4408
2,NBA 2016,colley,6018
3,NBA 2016,colley,9578
4,NBA 2016,colley,4537
...,...,...,...
115,NHL 2019,sr,3417
116,NHL 2019,sr,3063
117,NHL 2019,sr,5156
118,NHL 2019,sr,4022
